# 로그수익률 구하는 기법

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from pykrx import stock
from pykrx import bond
import warnings
warnings.filterwarnings('ignore')



In [2]:
총기법종목선정수익률 = pd.read_csv('./총기법종목수익률최종선정_10종목.csv')
코스피리스트 = pd.read_csv('./코스피A리스트.csv')

In [3]:
총기법종목선정수익률

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
0,20050401,20060331,"['엔씨소프트', '신도리코', '동서', '대덕', '퍼시스', 'KPX케미칼',...",0.261833,1.261833,유동성투자종목,2005
1,20060401,20070331,"['한화', '엔씨소프트', 'SBS', '동서', '신도리코', '퍼시스', 'K...",0.076401,1.434639,유동성투자종목,2006
2,20070401,20080331,"['엔씨소프트', '동서', '신도리코', '퍼시스', '모토닉', '삼영전자', ...",0.189146,1.895140,유동성투자종목,2007
3,20080401,20090331,"['SBS', '엔씨소프트', 'LS네트웍스', '동서', '신도리코', '퍼시스'...",-0.038545,1.783546,유동성투자종목,2008
4,20090401,20100331,"['CJ대한통운', '엔씨소프트', '에스원', '동서', '신도리코', 'SBS'...",0.226843,2.414974,유동성투자종목,2009
...,...,...,...,...,...,...,...
175,20180401,20190331,"['삼성에스디에스', '이마트', '현대글로비스', 'CJ', '에스원', '한샘'...",-0.090993,3.983032,활동성투자종목,2018
176,20190401,20200331,"['삼성에스디에스', '코웨이', '현대글로비스', '이마트', '에스원', 'CJ...",-0.373646,2.121140,활동성투자종목,2019
177,20200401,20210331,"['삼성에스디에스', '코웨이', '현대글로비스', '에스원', '이마트', 'LG...",0.492729,3.659016,활동성투자종목,2020
178,20210401,20220331,"['삼성에스디에스', '현대글로비스', '코웨이', '이마트', '에스원', 'GS...",-0.077048,3.300048,활동성투자종목,2021


In [4]:
코스피리스트['종목코드'] = 코스피리스트['code'].str.extract(r'(\d\d\d\d\d\d)')

In [5]:
총기법종목선정수익률['재무비율방식'].unique()

array(['유동성투자종목', '레버리지투자종목', '그레이엄투자종목', '린치투자종목', '버핏투자종목', '삼대투자종목',
       '성장성투자종목', '수익성투자종목', '시장가치투자종목', '활동성투자종목'], dtype=object)

In [6]:
레버리지기법 = 총기법종목선정수익률[총기법종목선정수익률['재무비율방식']=='레버리지투자종목']

In [7]:
레버리지기법

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
18,20050401,20060331,"['삼성전자', 'KT&G', '삼성SDI', '아모레G', '농심', '에스원',...",0.206773,1.206773,레버리지투자종목,2005
19,20060401,20070331,"['KT&G', 'NAVER', '아모레G', '롯데지주', '에스원', '엔씨소프...",0.005305,1.218480,레버리지투자종목,2006
20,20070401,20080331,"['KT&G', 'NAVER', '현대미포조선', '롯데지주', '롯데칠성', '에...",0.160676,1.574935,레버리지투자종목,2007
21,20080401,20090331,"['NAVER', 'KT&G', '현대미포조선', '아모레퍼시픽', '에스원', '...",-0.104648,1.305473,레버리지투자종목,2008
22,20090401,20100331,"['KT&G', '아모레퍼시픽', '엔씨소프트', '유한양행', '동서', '제일기...",0.353412,2.120253,레버리지투자종목,2009
23,20100401,20110331,"['KT&G', '아모레퍼시픽', '강원랜드', '엔씨소프트', '유한양행', '제...",0.276916,2.984301,레버리지투자종목,2010
24,20110401,20120331,"['LG화학', 'KT&G', '아모레퍼시픽', '엔씨소프트', '현대미포조선', ...",0.079461,3.300899,레버리지투자종목,2011
25,20120401,20130331,"['KT&G', '고려아연', '엔씨소프트', '아모레퍼시픽', '강원랜드', '한...",0.053564,3.531271,레버리지투자종목,2012
26,20130401,20140331,"['삼성전자', '현대모비스', 'NAVER', 'KT&G', '고려아연', '강원...",0.230629,4.576315,레버리지투자종목,2013
27,20140401,20150331,"['삼성전자', '현대모비스', 'NAVER', 'KT&G', '아모레퍼시픽', '...",0.204746,5.718040,레버리지투자종목,2014


In [8]:
레버리지기법['시작날짜']  = 레버리지기법['시작날짜'].astype(str)
레버리지기법['매도날짜']  = 레버리지기법['매도날짜'].astype(str)

In [9]:
시작날짜_list = 레버리지기법['시작날짜'].tolist()

In [10]:
시작날짜_list = [str(x) for x in 시작날짜_list]
시작날짜_list

['20050401',
 '20060401',
 '20070401',
 '20080401',
 '20090401',
 '20100401',
 '20110401',
 '20120401',
 '20130401',
 '20140401',
 '20150401',
 '20160401',
 '20170401',
 '20180401',
 '20190401',
 '20200401',
 '20210401',
 '20220401']

In [11]:
from datetime import datetime, timedelta
매수날짜_list = []
for 매수 in 시작날짜_list:
        
    날짜 = dt.datetime.strptime(매수,"%Y%m%d")
    Y = dt.datetime.strftime(날짜,'%Y')
    # M =dt.datetime.strftime(날짜,'%m')
    # D = dt.datetime.strftime(날짜,'%d')
    Y =int(Y)
    Y = str(Y)
    매수날짜 = Y+'0401'
    매수날짜_list.append(매수날짜)
매수날짜_list

['20050401',
 '20060401',
 '20070401',
 '20080401',
 '20090401',
 '20100401',
 '20110401',
 '20120401',
 '20130401',
 '20140401',
 '20150401',
 '20160401',
 '20170401',
 '20180401',
 '20190401',
 '20200401',
 '20210401',
 '20220401']

In [12]:
from datetime import datetime, timedelta
매도날짜_list = []
for 매도 in 시작날짜_list:
        
    날짜 = dt.datetime.strptime(매도,"%Y%m%d")
    Y = dt.datetime.strftime(날짜,'%Y')
    # M =dt.datetime.strftime(날짜,'%m')
    # D = dt.datetime.strftime(날짜,'%d')
    Y =int(Y)+1
    Y = str(Y)
    매도날짜 = Y+'0331'
    매도날짜_list.append(매도날짜)
매도날짜_list

['20060331',
 '20070331',
 '20080331',
 '20090331',
 '20100331',
 '20110331',
 '20120331',
 '20130331',
 '20140331',
 '20150331',
 '20160331',
 '20170331',
 '20180331',
 '20190331',
 '20200331',
 '20210331',
 '20220331',
 '20230331']

## 시작

In [17]:
stocks = 레버리지기법['포트폴리오종목명'].tolist()[12]
type(stocks)
stocks

"['삼성전자', 'NAVER', '현대모비스', '아모레퍼시픽', 'KT&G', '삼성에스디에스', '아모레G', '강원랜드', '고려아연', '코웨이']"

In [18]:
import ast
stocks = ast.literal_eval(stocks)

In [19]:
print(stocks) 
code_list = []

for 종목 in stocks:
    code_list.append(코스피리스트['종목코드'][코스피리스트['company']== 종목].tolist())
code_list = sum(code_list,[])
code_list

['삼성전자', 'NAVER', '현대모비스', '아모레퍼시픽', 'KT&G', '삼성에스디에스', '아모레G', '강원랜드', '고려아연', '코웨이']


['005930',
 '035420',
 '012330',
 '090430',
 '033780',
 '018260',
 '002790',
 '035250',
 '010130',
 '021240']

In [20]:
종가 = pd.DataFrame()
가중치 = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])  # 5종목이니까 자금의 20%씩
for ticker in code_list:
    df = stock.get_market_ohlcv(매수날짜_list[12], 매도날짜_list[12], ticker)
    종가 = pd.concat([종가,df['종가']],axis=1)
종가.columns = stocks
종가

#종가데이터로 수익률 계산하기.

# 일별수익률 = 종가.pct_change()
# 수익률 = pd.DataFrame()
# 수익률['일별수익률'] = (일별수익률['KT&G'] + 일별수익률['NAVER'] + 일별수익률['아모레G'] + 일별수익률['롯데지주'] + 일별수익률['에스원']) / 5
# 일별수익률
# 연간단순수익률 = np.sum(일별수익률.mean()*가중치)*250
# 수익률

# print(연간단순수익률)
# print(수익률.mean()*250)
# 시작날짜_list.append(시작날짜)
# 매도날짜_list.append(매도날짜)
# 포트폴리오종목명_list.append(포트폴리오종목명)
# 연간수익률_list.append(연간단순수익률)

,삼성전자,NAVER,현대모비스,아모레퍼시픽,KT&G,삼성에스디에스,아모레G,강원랜드,고려아연,코웨이
2017-04-03,41440,174848,239500,280000,98600,136000,121719,37400,418500,96800
2017-04-04,42080,170241,235500,283000,98600,137500,121719,37300,418000,96400
2017-04-05,42140,168839,229000,277500,99500,137500,122717,37250,417500,96200
2017-04-06,41840,168238,224000,276000,100000,135500,122717,36600,416500,95900
2017-04-07,41600,165835,224000,274000,101000,137500,123715,37000,420000,98600
...,...,...,...,...,...,...,...,...,...,...
2018-03-26,50280,158024,232500,296500,98500,248000,132694,27350,475500,90600
2018-03-27,49980,158024,245000,304500,98500,253000,135188,27550,483500,90700
2018-03-28,48700,157022,261500,304500,99100,245500,138181,27650,481000,89600
2018-03-29,49040,157423,254000,307000,100000,252500,136685,27250,482000,87800


In [21]:
종가['로그수익률1'] = np.log(종가.iloc[:,0]) - np.log(종가.iloc[:,0].shift(1))
종가['로그수익률2'] = np.log(종가.iloc[:,1]) - np.log(종가.iloc[:,1].shift(1))
종가['로그수익률3'] = np.log(종가.iloc[:,2]) - np.log(종가.iloc[:,2].shift(1))
종가['로그수익률4'] = np.log(종가.iloc[:,3]) - np.log(종가.iloc[:,3].shift(1))
종가['로그수익률5'] = np.log(종가.iloc[:,4]) - np.log(종가.iloc[:,4].shift(1))
종가['로그수익률6'] = np.log(종가.iloc[:,5]) - np.log(종가.iloc[:,5].shift(1))
종가['로그수익률7'] = np.log(종가.iloc[:,6]) - np.log(종가.iloc[:,6].shift(1))
종가['로그수익률8'] = np.log(종가.iloc[:,7]) - np.log(종가.iloc[:,7].shift(1))
종가['로그수익률9'] = np.log(종가.iloc[:,8]) - np.log(종가.iloc[:,8].shift(1))
종가['로그수익률10'] = np.log(종가.iloc[:,9]) - np.log(종가.iloc[:,9].shift(1))
종가

,삼성전자,NAVER,현대모비스,아모레퍼시픽,KT&G,삼성에스디에스,아모레G,강원랜드,고려아연,코웨이,로그수익률1,로그수익률2,로그수익률3,로그수익률4,로그수익률5,로그수익률6,로그수익률7,로그수익률8,로그수익률9,로그수익률10
2017-04-03,41440,174848,239500,280000,98600,136000,121719,37400,418500,96800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-04,42080,170241,235500,283000,98600,137500,121719,37300,418000,96400,0.015326,-0.026702,-0.016843,0.010657,0.000000,0.010969,0.000000,-0.002677,-0.001195,-0.004141
2017-04-05,42140,168839,229000,277500,99500,137500,122717,37250,417500,96200,0.001425,-0.008269,-0.027989,-0.019626,0.009086,0.000000,0.008166,-0.001341,-0.001197,-0.002077
2017-04-06,41840,168238,224000,276000,100000,135500,122717,36600,416500,95900,-0.007145,-0.003566,-0.022076,-0.005420,0.005013,-0.014652,0.000000,-0.017604,-0.002398,-0.003123
2017-04-07,41600,165835,224000,274000,101000,137500,123715,37000,420000,98600,-0.005753,-0.014386,0.000000,-0.007273,0.009950,0.014652,0.008100,0.010870,0.008368,0.027765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-26,50280,158024,232500,296500,98500,248000,132694,27350,475500,90600,0.011200,0.007636,-0.002148,0.027352,-0.002028,0.012171,0.018974,-0.001826,-0.003150,0.031393
2018-03-27,49980,158024,245000,304500,98500,253000,135188,27550,483500,90700,-0.005984,0.000000,0.052368,0.026624,0.000000,0.019961,0.018621,0.007286,0.016684,0.001103
2018-03-28,48700,157022,261500,304500,99100,245500,138181,27650,481000,89600,-0.025944,-0.006361,0.065176,0.000000,0.006073,-0.030093,0.021898,0.003623,-0.005184,-0.012202
2018-03-29,49040,157423,254000,307000,100000,252500,136685,27250,482000,87800,0.006957,0.002551,-0.029100,0.008177,0.009041,0.028114,-0.010885,-0.014572,0.002077,-0.020294


In [29]:
종가

,삼성전자,NAVER,현대모비스,아모레퍼시픽,KT&G,삼성에스디에스,아모레G,강원랜드,고려아연,코웨이,로그수익률1,로그수익률2,로그수익률3,로그수익률4,로그수익률5,로그수익률6,로그수익률7,로그수익률8,로그수익률9,로그수익률10
2017-04-03,41440,174848,239500,280000,98600,136000,121719,37400,418500,96800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-04,42080,170241,235500,283000,98600,137500,121719,37300,418000,96400,0.015326,-0.026702,-0.016843,0.010657,0.000000,0.010969,0.000000,-0.002677,-0.001195,-0.004141
2017-04-05,42140,168839,229000,277500,99500,137500,122717,37250,417500,96200,0.001425,-0.008269,-0.027989,-0.019626,0.009086,0.000000,0.008166,-0.001341,-0.001197,-0.002077
2017-04-06,41840,168238,224000,276000,100000,135500,122717,36600,416500,95900,-0.007145,-0.003566,-0.022076,-0.005420,0.005013,-0.014652,0.000000,-0.017604,-0.002398,-0.003123
2017-04-07,41600,165835,224000,274000,101000,137500,123715,37000,420000,98600,-0.005753,-0.014386,0.000000,-0.007273,0.009950,0.014652,0.008100,0.010870,0.008368,0.027765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-26,50280,158024,232500,296500,98500,248000,132694,27350,475500,90600,0.011200,0.007636,-0.002148,0.027352,-0.002028,0.012171,0.018974,-0.001826,-0.003150,0.031393
2018-03-27,49980,158024,245000,304500,98500,253000,135188,27550,483500,90700,-0.005984,0.000000,0.052368,0.026624,0.000000,0.019961,0.018621,0.007286,0.016684,0.001103
2018-03-28,48700,157022,261500,304500,99100,245500,138181,27650,481000,89600,-0.025944,-0.006361,0.065176,0.000000,0.006073,-0.030093,0.021898,0.003623,-0.005184,-0.012202
2018-03-29,49040,157423,254000,307000,100000,252500,136685,27250,482000,87800,0.006957,0.002551,-0.029100,0.008177,0.009041,0.028114,-0.010885,-0.014572,0.002077,-0.020294


In [23]:
종가['일별로그수익률'] = (종가['로그수익률1'] + 종가['로그수익률2'] + 종가['로그수익률3'] + 종가['로그수익률4'] + 종가['로그수익률5']+
                종가['로그수익률6'] + 종가['로그수익률7'] + 종가['로그수익률8'] + 종가['로그수익률9'] + 종가['로그수익률10']) / 10
종가

,삼성전자,SK하이닉스,셀트리온,현대모비스,LG생활건강,크래프톤,LG,고려아연,KT&G,삼성에스디에스,...,로그수익률2,로그수익률3,로그수익률4,로그수익률5,로그수익률6,로그수익률7,로그수익률8,로그수익률9,로그수익률10,일별로그수익률
2022-04-01 00:00:00,69100,116000,169000,217000,896000,278000,76200,590000,81000,139500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-04 00:00:00,69300,117000,170000,215000,904000,298500,75500,602000,81400,139000,...,0.008584,0.005900,-0.009259,0.008889,0.071149,-0.009229,0.020135,0.004926,-0.003591,0.010039
2022-04-05 00:00:00,69200,116500,171500,217000,909000,293500,75300,605000,81400,140000,...,-0.004283,0.008785,0.009259,0.005516,-0.016892,-0.002653,0.004971,0.000000,0.007168,0.001043
2022-04-06 00:00:00,68500,113000,168500,216000,895000,287000,74800,631000,81200,139000,...,-0.030503,-0.017648,-0.004619,-0.015521,-0.022395,-0.006662,0.042077,-0.002460,-0.007168,-0.007507
2022-04-07 00:00:00,68000,113500,166500,212000,865000,284000,73800,630000,81400,136500,...,0.004415,-0.011940,-0.018692,-0.034094,-0.010508,-0.013459,-0.001586,0.002460,-0.018149,-0.010888
2022-04-08 00:00:00,67800,112000,164000,209000,868000,280000,74500,628000,81800,137000,...,-0.013304,-0.015129,-0.014252,0.003462,-0.014185,0.009440,-0.003180,0.004902,0.003656,-0.004153
2022-04-11 00:00:00,67900,111500,162000,208000,881000,281500,74900,619000,82300,136500,...,-0.004474,-0.012270,-0.004796,0.014866,0.005343,0.005355,-0.014435,0.006094,-0.003656,-0.000650
2022-04-12 00:00:00,67000,111000,161000,206500,882000,267000,72900,617000,82000,136000,...,-0.004494,-0.006192,-0.007238,0.001134,-0.052884,-0.027065,-0.003236,-0.003652,-0.003670,-0.012064
2022-04-13 00:00:00,68700,113000,164500,210000,890000,267500,74800,625000,82000,138000,...,0.017858,0.021506,0.016807,0.009029,0.001871,0.025729,0.012883,0.000000,0.014599,0.014534
2022-04-14 00:00:00,67500,110000,164000,207500,894000,266500,74800,634000,80800,137500,...,-0.026907,-0.003044,-0.011976,0.004484,-0.003745,0.000000,0.014297,-0.014742,-0.003630,-0.006289


In [24]:
일별로그수익률_2022 = 종가['일별로그수익률']

In [25]:
일별로그수익률_2022

2022-04-01         NaN
2022-04-04    0.010039
2022-04-05    0.001043
2022-04-06   -0.007507
2022-04-07   -0.010888
2022-04-08   -0.004153
2022-04-11   -0.000650
2022-04-12   -0.012064
2022-04-13    0.014534
2022-04-14   -0.006289
2022-04-15   -0.014528
2022-04-18    0.006161
2022-04-19    0.012343
2022-04-20    0.000856
2022-04-21    0.001458
2022-04-22   -0.010003
2022-04-25   -0.021045
2022-04-26    0.008454
2022-04-27   -0.008686
2022-04-28    0.018787
2022-04-29    0.009713
2022-05-02   -0.002091
2022-05-03   -0.006505
2022-05-04   -0.003745
2022-05-06   -0.013734
2022-05-09   -0.011615
2022-05-10   -0.004799
2022-05-11    0.000791
2022-05-12   -0.029275
2022-05-13    0.014738
2022-05-16   -0.009808
Name: 일별로그수익률, dtype: float64

In [264]:
일별로그수익률_list = [일별로그수익률_2005,
일별로그수익률_2006,
일별로그수익률_2007,
일별로그수익률_2008,
일별로그수익률_2009,
일별로그수익률_2010,
일별로그수익률_2011,
일별로그수익률_2012,
일별로그수익률_2013,
일별로그수익률_2014,
일별로그수익률_2015,
일별로그수익률_2016,
일별로그수익률_2017,
일별로그수익률_2018,
일별로그수익률_2019,
일별로그수익률_2020,
일별로그수익률_2021,
일별로그수익률_2022]

In [268]:
레버리지로그수익률 = pd.DataFrame()
for 로그 in 일별로그수익률_list:
    
    레버리지로그수익률 = pd.concat([레버리지로그수익률,로그],axis=0)
    
레버리지로그수익률

,0
2005-04-01,NaN
2005-04-04,-0.004507
2005-04-06,0.015333
2005-04-07,-0.006284
2005-04-08,0.001610
...,...
2022-05-09,-0.019313
2022-05-10,0.004609
2022-05-11,0.004382
2022-05-12,-0.040458


In [312]:
test = 일별로그수익률_2020.copy()
test.isna().sum()

1

In [309]:
test = test.fillna(0)

In [314]:
np.sum(test.mean())*250

1.6766323930338058

In [313]:
test.iloc[0]  = 1
test

2020-04-01    1.000000
2020-04-02    0.031864
2020-04-03    0.006671
2020-04-06    0.037632
2020-04-07    0.012749
                ...   
2021-03-25   -0.002934
2021-03-26   -0.008825
2021-03-29    0.006947
2021-03-30    0.001862
2021-03-31    0.001780
Name: 일별로그수익률, Length: 246, dtype: float64

In [315]:
누적수익률= (1+test).cumprod()-1 
누적수익률

2020-04-01    1.000000
2020-04-02    1.063728
2020-04-03    1.077495
2020-04-06    1.155676
2020-04-07    1.183158
                ...   
2021-03-25    2.677145
2021-03-26    2.644694
2021-03-29    2.670015
2021-03-30    2.676848
2021-03-31    2.683394
Name: 일별로그수익률, Length: 246, dtype: float64

In [269]:
레버리지로그수익률.columns=['레버리지기법']

In [272]:
# 레버리지로그수익률.to_csv('레버리지로그수익률.csv',index=True)

In [339]:
시장가치로그수익률 = pd.read_csv('./시장가치로그수익률.csv')
레버리지로그수익률 = pd.read_csv('./레버리지로그수익률.csv')
그레이엄로그수익률 = pd.read_csv('./그레이엄로그수익률.csv')

In [341]:
레버리지로그수익률

,Unnamed: 0,레버리지기법
0,2005-04-01,NaN
1,2005-04-04,-0.004507
2,2005-04-06,0.015333
3,2005-04-07,-0.006284
4,2005-04-08,0.001610
...,...,...
4229,2022-05-09,-0.019313
4230,2022-05-10,0.004609
4231,2022-05-11,0.004382
4232,2022-05-12,-0.040458


In [338]:
레버리지로그수익률.tail(32)

,Unnamed: 0,레버리지기법
4202,2022-03-30,0.010743
4203,2022-03-31,-0.003213
4204,2022-04-01,NaN
4205,2022-04-04,-0.002709
4206,2022-04-05,0.001861
4207,2022-04-06,-0.014383
4208,2022-04-07,-0.015570
4209,2022-04-08,-0.001718
4210,2022-04-11,0.000198
4211,2022-04-12,-0.007272


In [336]:
시장가치로그수익률

,Unnamed: 0,일별로그수익률
0,2005-04-01,NaN
1,2005-04-04,-0.013264
2,2005-04-06,0.002318
3,2005-04-07,-0.008626
4,2005-04-08,-0.008570
...,...,...
4199,2022-03-25,-0.006623
4200,2022-03-28,-0.002587
4201,2022-03-29,-0.006926
4202,2022-03-30,-0.004540


In [327]:
로그수익률 = pd.merge(레버리지로그수익률,시장가치로그수익률, how='outer',on='Unnamed: 0')

In [330]:
로그수익률

,Unnamed: 0,레버리지기법,일별로그수익률
0,2005-04-01,NaN,NaN
1,2005-04-04,-0.004507,-0.013264
2,2005-04-06,0.015333,0.002318
3,2005-04-07,-0.006284,-0.008626
4,2005-04-08,0.001610,-0.008570
...,...,...,...
4476,2022-05-09,-0.019313,NaN
4477,2022-05-10,0.004609,NaN
4478,2022-05-11,0.004382,NaN
4479,2022-05-12,-0.040458,NaN


In [331]:
로그수익률[로그수익률['일별로그수익률'].isna()].

,Unnamed: 0,레버리지기법,일별로그수익률
0,2005-04-01,NaN,NaN
252,2006-04-03,NaN,NaN
498,2007-04-02,NaN,NaN
743,2008-04-01,NaN,NaN
991,2009-04-01,NaN,NaN
1244,2010-04-01,NaN,NaN
1494,2011-04-01,NaN,NaN
1744,2012-04-02,NaN,NaN
1991,2013-04-01,NaN,NaN
2238,2014-04-01,NaN,NaN


In [23]:
총기법종목선정수익률

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
0,20050401,20060331,"['엔씨소프트', '신도리코', '동서', '대덕', '퍼시스']",-0.768626,1.231374,유동성종목,2005
1,20060401,20070331,"['한화', '엔씨소프트', 'SBS', '동서', '신도리코']",0.086176,1.423665,유동성종목,2006
2,20070401,20080331,"['엔씨소프트', '동서', '신도리코', '퍼시스', '모토닉']",0.105855,1.680222,유동성종목,2007
3,20080401,20090331,"['SBS', '엔씨소프트', 'LS네트웍스', '동서', '신도리코']",0.058882,1.838038,유동성종목,2008
4,20090401,20100331,"['CJ대한통운', '엔씨소프트', '에스원', '동서', '신도리코']",0.161794,2.297215,유동성종목,2009
...,...,...,...,...,...,...,...
175,20180401,20190331,"['삼성에스디에스', '이마트', '현대글로비스', 'CJ', '에스원']",-0.156100,3.035902,활동성종목,2018
176,20190401,20200331,"['삼성에스디에스', '코웨이', '현대글로비스', '이마트', '에스원']",-0.350214,1.622473,활동성종목,2019
177,20200401,20210331,"['삼성에스디에스', '코웨이', '현대글로비스', '에스원', '이마트']",0.458164,2.823996,활동성종목,2020
178,20210401,20220331,"['삼성에스디에스', '현대글로비스', '코웨이', '이마트', '에스원']",-0.081601,2.511952,활동성종목,2021


In [26]:
총기법종목선정수익률['재무비율방식'].unique()

array(['유동성투자종목', '레버리지투자종목', '그레이엄투자종목', '린치투자종목', '버핏투자종목', '삼대투자종목',
       '성장성투자종목', '수익성투자종목', '시장가치투자종목', '활동성투자종목'], dtype=object)

# 로그수익률10종목 함수

In [54]:
from datetime import datetime, timedelta
import ast

def 로그수익률함수():
    
    전체로그수익률 = pd.DataFrame()

    기법 = 총기법종목선정수익률[총기법종목선정수익률['재무비율방식']=='활동성투자종목']
    기법['시작날짜']  = 기법['시작날짜'].astype(str)
    기법['매도날짜']  = 기법['매도날짜'].astype(str)
    시작날짜_list = 기법['시작날짜'].tolist()
    시작날짜_list = [str(x) for x in 시작날짜_list]

    매수날짜_list = []
    for 매수 in 시작날짜_list:

        날짜 = dt.datetime.strptime(매수,"%Y%m%d")
        Y = dt.datetime.strftime(날짜,'%Y')
        # M =dt.datetime.strftime(날짜,'%m')
        # D = dt.datetime.strftime(날짜,'%d')
        Y =int(Y)
        Y = str(Y)
        매수날짜 = Y+'0401'
        매수날짜_list.append(매수날짜)
    매수날짜_list
    # -----------------------------------------------------------------------------
    매도날짜_list = []
    for 매도 in 시작날짜_list:

        날짜 = dt.datetime.strptime(매도,"%Y%m%d")
        Y = dt.datetime.strftime(날짜,'%Y')
        # M =dt.datetime.strftime(날짜,'%m')
        # D = dt.datetime.strftime(날짜,'%d')
        Y =int(Y)+1
        Y = str(Y)
        매도날짜 = Y+'0331'
        매도날짜_list.append(매도날짜)
    매도날짜_list
    # -----------------------------------------------------------------------------
    # len(기법['포트폴리오종목명'].tolist())

    for i , e in enumerate(기법['포트폴리오종목명'].tolist()):
        stocks = 기법['포트폴리오종목명'].tolist()[i]  #i번쨰
        stocks = ast.literal_eval(stocks)

        code_list = []

        for 종목 in stocks:
            code_list.append(코스피리스트['종목코드'][코스피리스트['company']== 종목].tolist())
        code_list = sum(code_list,[])

        종가 = pd.DataFrame()
        가중치 = np.array([0.2,0.2,0.2,0.2,0.2])  # 5종목이니까 자금의 20%씩
        for ticker in code_list:
            df = stock.get_market_ohlcv(매수날짜_list[i], 매도날짜_list[i], ticker)
            종가 = pd.concat([종가,df['종가']],axis=1)
        종가.columns = stocks

        종가['로그수익률1'] = np.log(종가.iloc[:,0]) - np.log(종가.iloc[:,0].shift(1))
        종가['로그수익률2'] = np.log(종가.iloc[:,1]) - np.log(종가.iloc[:,1].shift(1))
        종가['로그수익률3'] = np.log(종가.iloc[:,2]) - np.log(종가.iloc[:,2].shift(1))
        종가['로그수익률4'] = np.log(종가.iloc[:,3]) - np.log(종가.iloc[:,3].shift(1))
        종가['로그수익률5'] = np.log(종가.iloc[:,4]) - np.log(종가.iloc[:,4].shift(1))
        종가['로그수익률6'] = np.log(종가.iloc[:,5]) - np.log(종가.iloc[:,5].shift(1))
        종가['로그수익률7'] = np.log(종가.iloc[:,6]) - np.log(종가.iloc[:,6].shift(1))
        종가['로그수익률8'] = np.log(종가.iloc[:,7]) - np.log(종가.iloc[:,7].shift(1))
        종가['로그수익률9'] = np.log(종가.iloc[:,8]) - np.log(종가.iloc[:,8].shift(1))
        종가['로그수익률10'] = np.log(종가.iloc[:,9]) - np.log(종가.iloc[:,9].shift(1))
        

        종가['일별로그수익률'] = (종가['로그수익률1'] + 종가['로그수익률2'] + 종가['로그수익률3'] + 종가['로그수익률4'] + 종가['로그수익률5']+
                종가['로그수익률6'] + 종가['로그수익률7'] + 종가['로그수익률8'] + 종가['로그수익률9'] + 종가['로그수익률10']) / 10

        일별로그수익률 = 종가['일별로그수익률']

        전체로그수익률 = pd.concat([전체로그수익률,일별로그수익률],axis=0)


    전체로그수익률.columns=['활동성기법']
    
    return 전체로그수익률
        
    


In [55]:
전체로그수익률 = 로그수익률함수()
전체로그수익률

,활동성기법
2005-04-01,NaN
2005-04-04,-0.001492
2005-04-06,0.017410
2005-04-07,-0.009234
2005-04-08,0.006397
...,...
2022-05-10,-0.015276
2022-05-11,0.018093
2022-05-12,0.005001
2022-05-13,0.010580


In [56]:
활동성로그수익률 = 전체로그수익률

In [57]:
로그수익률_list  = [유동성로그수익률,
레버리지로그수익률,           
그레이엄로그수익률,   
린치로그수익률, 
버핏로그수익률,  
삼대투자로그수익률,
성장성로그수익률, 
수익성로그수익률,
시장가치로그수익률,
활동성로그수익률]

In [58]:
로그수익률전체 = pd.concat(로그수익률_list,axis=1)

In [59]:
로그수익률전체

,유동성기법,레버리지기법,그레이엄기법,린치기법,버핏기법,삼대투자기법,성장성기법,수익성기법,시장가치기법,활동성기법
2005-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-04,-0.001885,-0.003509,0.003417,0.015300,-0.004737,-0.000839,-0.000324,-0.002091,-0.012773,-0.001492
2005-04-06,-0.005572,0.012566,0.005841,-0.002118,0.011094,0.000858,0.002884,0.007565,-0.000898,0.017410
2005-04-07,-0.002721,-0.010705,-0.004159,-0.001387,-0.001534,-0.002636,0.001947,0.000370,0.000927,-0.009234
2005-04-08,-0.002758,-0.003010,-0.001109,-0.000854,0.002285,-0.004228,0.010989,-0.000463,0.009500,0.006397
...,...,...,...,...,...,...,...,...,...,...
2022-05-10,-0.015892,-0.004799,-0.002160,-0.009955,-0.010384,-0.003993,-0.004541,-0.006504,-0.001658,-0.015276
2022-05-11,0.018021,0.000791,-0.003038,0.012060,0.009773,0.009442,0.009268,0.013275,-0.007461,0.018093
2022-05-12,-0.024158,-0.029275,-0.013050,-0.001186,-0.011456,-0.006010,-0.020785,-0.035540,-0.003394,0.005001
2022-05-13,0.031105,0.014738,0.018195,0.017261,0.022215,0.012290,0.025838,0.014019,0.015668,0.010580


In [60]:
로그수익률전체.to_csv('로그수익률전체_10종목.csv',index=True)

In [47]:
전체로그수익률

""


In [22]:
매수날짜_list[1]

'20060401'

In [21]:
for i , e in enumerate(기법['포트폴리오종목명'].tolist()):
    print(i,e)

0 ['삼성전자', 'KT&G', '삼성SDI', '아모레G', '농심']
1 ['KT&G', 'NAVER', '아모레G', '롯데지주', '에스원']
2 ['KT&G', 'NAVER', '현대미포조선', '롯데지주', '롯데칠성']
3 ['NAVER', 'KT&G', '현대미포조선', '아모레퍼시픽', '에스원']
4 ['KT&G', '아모레퍼시픽', '엔씨소프트', '유한양행', '동서']
5 ['KT&G', '아모레퍼시픽', '강원랜드', '엔씨소프트', '유한양행']
6 ['LG화학', 'KT&G', '아모레퍼시픽', '엔씨소프트', '현대미포조선']
7 ['KT&G', '고려아연', '엔씨소프트', '아모레퍼시픽', '강원랜드']
8 ['삼성전자', '현대모비스', 'NAVER', 'KT&G', '고려아연']
9 ['삼성전자', '현대모비스', 'NAVER', 'KT&G', '아모레퍼시픽']
10 ['삼성전자', 'NAVER', '삼성에스디에스', '아모레퍼시픽', '기아']
11 ['현대모비스', '아모레퍼시픽', 'NAVER', 'SK하이닉스', 'KT&G']
12 ['삼성전자', 'NAVER', '현대모비스', '아모레퍼시픽', 'KT&G']
13 ['삼성전자', 'SK하이닉스', 'NAVER', 'LG생활건강', '아모레퍼시픽']
14 ['삼성전자', 'SK하이닉스', 'LG생활건강', 'NAVER', '삼성에스디에스']
15 ['삼성전자', 'NAVER', 'LG생활건강', '엔씨소프트', '삼성에스디에스']
16 ['삼성전자', 'NAVER', '셀트리온', 'LG생활건강', '엔씨소프트']
17 ['삼성전자', 'SK하이닉스', '셀트리온', '현대모비스', 'LG생활건강']
